<a href="https://colab.research.google.com/github/OsheenArya/SpeechEmotionRecognitionCapstone/blob/main/New_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [3]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
#Extract paths of the datasets

ravdess_emotions={'01':'neutral', '02':'calm', '03':'happy', '04':'sad',
                  '05':'angry', '06':'fear', '07':'disgust', '08':'suprise'}

crema_emotions = {'NEU': 'neutral', 'HAP':'happy', 'SAD':'sad', 'ANG':'angry', 'FEA':'fear','DIS':'disgust'}

savee_emotions = {'a':'angry', 'h':'happy', 's':'sad', 'n':'neutral', 'f':'fear', 'd':'disgust'}

data_path = '/gdrive/MyDrive/speech_datasets/'
processed_data = []

for root, dirs, files in os.walk(data_path):
  for file in files:
    file_path = os.path.join(root,file)

    dataset = root.split('/')[4]

    if dataset == 'Crema':
      emotion = crema_emotions[file.split('_')[2]]
    elif dataset == 'Tess':
      emotion = file.split('_')[2].split('.')[0]
      if emotion == 'ps':
        emotion = 'suprise'
    elif dataset == 'Ravdess':
      emotion = ravdess_emotions[file.split("-")[2]]
    else:
      emotion_code = file.split('_')[1][:2]
      if emotion_code.__contains__('su'):
        emotion = 'suprise'
      else:
        emotion = savee_emotions[emotion_code[0]]

    processed_data.append([file_path, dataset, emotion])


speech_data = pd.DataFrame(processed_data, columns=['file_name','dataset','emotion'])


In [5]:
speech_data


,file_name,dataset,emotion
0,/gdrive/MyDrive/speech_datasets/Crema/1080_IEO...,Crema,happy
1,/gdrive/MyDrive/speech_datasets/Crema/1080_IEO...,Crema,fear
2,/gdrive/MyDrive/speech_datasets/Crema/1080_DFA...,Crema,happy
3,/gdrive/MyDrive/speech_datasets/Crema/1079_WSI...,Crema,sad
4,/gdrive/MyDrive/speech_datasets/Crema/1079_WSI...,Crema,fear
...,...,...,...
12157,/gdrive/MyDrive/speech_datasets/Tess/OAF_happy...,Tess,happy
12158,/gdrive/MyDrive/speech_datasets/Tess/OAF_happy...,Tess,happy
12159,/gdrive/MyDrive/speech_datasets/Tess/OAF_happy...,Tess,happy
12160,/gdrive/MyDrive/speech_datasets/Tess/OAF_happy...,Tess,happy


In [6]:
from librosa import feature

def extract_feature(file_name, mfcc, chroma, mel):
  X, sample_rate = librosa.load(os.path.join(file_name), res_type = "kaiser_fast")
  result = np.array([])
  if mfcc:
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    result = np.hstack((result, mfccs))
  if chroma:
    stft =np.abs(librosa.stft(X))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
  if mel:
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
   

  return result



In [7]:
file_names = speech_data['file_name'].tolist()

def get_features(file_path):
  features = []
  for file in file_path:
    feature = extract_feature(file, mfcc=True , chroma=True, mel=True)
    features.append(feature)
  
  return pd.DataFrame(np.array(features))



In [ ]:
feature_data = get_features(file_names)

In [10]:
labels = speech_data['emotion']
feature_data['emotion'] = label

In [16]:
feature_data.to_csv('feature.csv')

In [18]:
from sklearn.model_selection import train_test_split

X = feature_data.drop(columns= 'emotion')
y = feature_data['emotion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)


In [22]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(200,), learning_rate='adaptive', max_iter=10000)

model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)


In [23]:
accuracy

0.5195232223592273